In [ ]:
import torch
import torch.nn as nn

class CoAttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(CoAttentionLayer, self).__init__()
        self.hidden_size = hidden_size
        self.W_b = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, X1, X2):
        # X1 é a primeira sequência de entrada de tamanho [batch_size, seq_len1, hidden_size]
        # X2 é a segunda sequência de entrada de tamanho [batch_size, seq_len2, hidden_size]
        
        # aplicar a transformação W_b em X2 para obter Y de tamanho [batch_size, seq_len2, hidden_size]
        Y = self.W_b(X2)
        
        # calcular a atenção entre X1 e Y
        attention_scores = torch.bmm(X1, Y.transpose(1, 2))
        attention_weights = self.softmax(attention_scores)
        
        # calcular a atenção entre Y e X1
        Y_t = Y.transpose(1, 2)
        co_attention_scores = torch.bmm(attention_weights, Y_t)
        co_attention_weights = self.softmax(co_attention_scores)
        
        # calcular as representações atencionais finais
        X1_t = X1.transpose(1, 2)
        attended_X1 = torch.bmm(co_attention_weights, X1_t).transpose(1, 2)
        attended_Y = torch.bmm(attention_weights, Y).transpose(1, 2)
        
        # concatenar as representações atencionais e retornar
        concatenated = torch.cat((attended_X1, attended_Y), dim=2)
        return concatenated
